In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score

In [ ]:
df=pd.read_csv('https://raw.githubusercontent.com/abhifg/Big-Mart-Sales-Prediction/main/preprocessed.csv')


In [ ]:
df.head()

,Item_Weight,Item_Visibility,Item_MRP,Item_Outlet_Sales,Outlet_Age,Item_Fat_Content_Regular,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,Item_Type_Frozen Foods,Item_Type_Fruits and Vegetables,Item_Type_Hard Drinks,Item_Type_Health and Hygiene,Item_Type_Household,Item_Type_Meat,Item_Type_Others,Item_Type_Seafood,Item_Type_Snack Foods,Item_Type_Soft Drinks,Item_Type_Starchy Foods,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,9.30,0.016047,249.8092,3735.1380,22.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,5.92,0.019278,48.2692,443.4228,12.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
2,17.50,0.016760,141.6180,2097.2700,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,19.20,0.000000,182.0950,732.3800,23.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,8.93,0.000000,53.8614,994.7052,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


# **Transformation into Gaussian Distribution**

In [ ]:

df['Item_Visibility'] = df['Item_Visibility']**(1/3)
df['Item_Outlet_Sales'] = df['Item_Outlet_Sales']**(1/8)

In [ ]:
x=df.drop('Item_Outlet_Sales',axis='columns')
y=df['Item_Outlet_Sales']

In [ ]:
x.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Age,Item_Fat_Content_Regular,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,Item_Type_Frozen Foods,Item_Type_Fruits and Vegetables,Item_Type_Hard Drinks,Item_Type_Health and Hygiene,Item_Type_Household,Item_Type_Meat,Item_Type_Others,Item_Type_Seafood,Item_Type_Snack Foods,Item_Type_Soft Drinks,Item_Type_Starchy Foods,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,9.30,0.252232,249.8092,22.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,5.92,0.268136,48.2692,12.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
2,17.50,0.255913,141.6180,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,19.20,0.000000,182.0950,23.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,8.93,0.000000,53.8614,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [ ]:
y.head()

0    2.796007
1    2.142162
2    2.601398
3    2.280825
4    2.369801
Name: Item_Outlet_Sales, dtype: float64

## **Model Fitting**

## **1. Linear Regression**

In [ ]:
from sklearn.linear_model import LinearRegression
lir=LinearRegression()
scores_lir=cross_val_score(lir,X=x,y=y,cv=5)

In [ ]:
print('Score for Linear Regression is: ',scores_lir.mean())

Score for Linear Regression is:  0.6973285104745223


## **2. Lasso Regression**

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
param={
    'alpha':[0.0001,0.0005,0.0009,0.01,0.05,0.09,0.1,0.5,0.9]
}
lasso_reg=GridSearchCV(Lasso(),param_grid=param,cv=5,n_jobs=-1,verbose=1,return_train_score=False)
lasso_reg.fit(x,y)
print('Best Score: ',lasso_reg.best_score_)
print('Best Parameter: ',lasso_reg.best_params_)


Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Best Score:  0.6975298504386716
Best Parameter:  {'alpha': 0.0001}


[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    1.8s finished


## **3. Ridge Regression**

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
param={
    'alpha':[0.0001,0.0005,0.0009,0.01,0.05,0.09,0.1,0.5,0.9]
}
ridge_reg=GridSearchCV(Ridge(),param_grid=param,cv=5,n_jobs=-1,verbose=1,return_train_score=False)
ridge_reg.fit(x,y)
print('Best Score: ',ridge_reg.best_score_)
print('Best Parameter: ',ridge_reg.best_params_)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best Score:  0.6973292874729211
Best Parameter:  {'alpha': 0.1}


[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    0.3s finished


## **4. XGBOOST**

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
xgb_params={
    'n_estimators':[12,13,15,16,17,18,19,20,22,24,25],
    'max_depth':[4,5,6,7],
    'booster':['gbtree'],
    'min_child_weight':[60,65,72,75,76,78,80],
    'base_score':[3,4,5,6,7,8],
    'lambda':[2,4,6,8,10,12],
    'alpha':[3,6,8,9,12,14]
}
xgb=RandomizedSearchCV(XGBRegressor(),param_distributions=xgb_params,cv=8,n_iter=100,n_jobs=-1,verbose=2,return_train_score=False)
xgb.fit(X=x,y=y)

Fitting 8 folds for each of 100 candidates, totalling 800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:   27.3s
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:  2.3min finished


[16:01:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


RandomizedSearchCV(cv=8, error_score=nan,
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, gamma=0,
                                          importance_type='gain',
                                          learning_rate=0.1, max_delta_step=0,
                                          max_depth=3, min_child_weight=1,
                                          missing=None, n_estimators=100,
                                          n_jobs=1, nthread=None,
                                          objective='reg:linear',
                                          random_state=0, reg_alpha=...
                   iid='deprecated', n_iter=100, n_jobs=-1,
                   param_distributions={'alpha': [3, 6, 8, 9, 12, 14],
                                        'base_score': [3,

In [ ]:
print('Best Score: ',xgb.best_score_)
print('Best Paratmeter: ',xgb.best_params_)

Best Score:  0.695444122946272
Best Paratmeter:  {'n_estimators': 25, 'min_child_weight': 72, 'max_depth': 5, 'lambda': 4, 'booster': 'gbtree', 'base_score': 3, 'alpha': 14}


# **5. Random Forest Regression**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_params={
    'n_estimators':[100,150,200,250],
    'max_depth': [5,6,7,8],
    'min_samples_split': [8,9,10,11],
    'min_samples_leaf': [20,22,24,25]
}
rf=RandomizedSearchCV(RandomForestRegressor(),param_distributions=rf_params,n_iter=100,n_jobs=-1,verbose=2,return_train_score=False)
rf.fit(X=x,y=y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed: 11.6min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 15.7min finished


RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                           

In [ ]:
print("R2 score: ",rf.best_score_)
print("Best Parameter: ",rf.best_params_)

R2 score:  0.7147429577723148
Best Parameter:  {'n_estimators': 200, 'min_samples_split': 9, 'min_samples_leaf': 20, 'max_depth': 6}


# **Best Model**

From the above model We can clearly see that Random Forest Regression method gives the highest R2 score 71.47% among all the models.

So for this dataset Random Forest Regression with {n_estimators: 250, min_samples_split: 9, min_samples_leaf: 22, max_depth: 6} is the best model.

In [ ]:
import pickle
file=open('random_forest_model.pkl','wb')
pickle.dump(rf,file)

In [ ]:
rf.predict([[5.92,0.057,48.2692,12.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0]])

array([2.25391159])